In [ ]:
import numpy as np
import random
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Set style untuk visualisasi
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# DEFINISI MASALAH & FUNGSI
# ============================================================================

# Sistem Persamaan Linier: Ax = b
# A (coefficients): Matriks koefisien 8x8
coefficients = np.array([
    [-3, 2, -3, -3, 0, 2, -2, 2],
    [-1, -2, -3, -4, 2, -3, 2, 5],
    [5, -3, 4, 4, 4, -1, 4, -2],
    [-2, 1, 4, -1, -3, 0, -1, 5],
    [3, 0, -4, -4, -3, -1, 3, 4],
    [-3, -4, 4, -1, -4, -2, 1, -3],
    [-2, 5, -1, 4, -2, -3, -3, 3],
    [3, 1, 4, 0, 0, -2, 4, -1]
])

# b (results): Vektor hasil
results = np.array([-51, -53, 105, -61, 14, -51, -47, 46])

# Solusi yang benar (untuk tujuan evaluasi)
true_solution = np.array([10, 1, -2, 7, 1, 8, 9, -3])

def calculate_error(x):
    """Menghitung Sum Squared Error (SSE) dari solusi x."""
    predictions = np.dot(coefficients, x)
    errors = predictions - results
    # Fungsi objektif: meminimalkan SSE
    return np.sum(errors ** 2)

def calculate_fitness(error):
    """Menghitung fitness (kualitas) solusi dari error."""
    # Fitness = 1 / (1 + Error), untuk mengubah masalah minimasi menjadi maksimasi
    return 1.0 / (1.0 + error)

# ============================================================================
# CLASS ACO (ANT COLONY OPTIMIZER)
# ============================================================================

class AntColonyOptimizer:
    def _init_(self, n_ants=30, n_iterations=200, evaporation_rate=0.1, 
                 alpha=1.0, beta=2.0, search_range=(-15, 15)):
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.search_range = search_range
        self.n_vars = 8
        self.pheromone = np.ones(self.n_vars)
        self.best_solution = None
        self.best_fitness = 0
        self.best_error = float('inf')
        
    def construct_solution(self):
        """Membuat solusi baru berdasarkan nilai feromon (menggunakan distribusi normal)."""
        solution = np.zeros(self.n_vars)
        for i in range(self.n_vars):
            # Jika belum ada solusi terbaik, gunakan feromon sebagai mean, jika tidak, gunakan solusi terbaik
            mean = self.pheromone[i] if self.best_solution is None else self.best_solution[i]
            # Standar deviasi inversely proporsional terhadap feromon (lebih banyak feromon = lebih spesifik)
            std = 5.0 / (1.0 + self.pheromone[i])
            
            # Ambil sampel dari distribusi normal
            solution[i] = np.random.normal(mean, std)
            
            # Batasi solusi dalam rentang pencarian
            solution[i] = np.clip(solution[i], self.search_range[0], self.search_range[1])
        return solution
    
    def update_pheromone(self, all_solutions, all_fitness):
        """Memperbarui feromon melalui penguapan dan pengendapan."""
        # 1. Penguapan (Evaporation)
        self.pheromone *= (1 - self.evaporation_rate)
        
        # 2. Pengendapan (Deposition)
        for solution, fitness in zip(all_solutions, all_fitness):
            # Deposit feromon proporsional terhadap fitness
            deposit = fitness * 10
            for i in range(self.n_vars):
                # Nilai deposit feromon lebih tinggi jika solusi dekat dengan solusi terbaik
                if self.best_solution is not None:
                    similarity = 1.0 / (1.0 + abs(solution[i] - self.best_solution[i]))
                    self.pheromone[i] += deposit * similarity
                else:
                    self.pheromone[i] += deposit
    
    def optimize(self, verbose=False):
        """Menjalankan proses optimasi ACO."""
        for iteration in range(self.n_iterations):
            all_solutions = []
            all_fitness = []
            all_errors = []
            
            # Fase Konstruksi Semut
            for ant in range(self.n_ants):
                solution = self.construct_solution()
                error = calculate_error(solution)
                fitness = calculate_fitness(error)
                
                all_solutions.append(solution)
                all_fitness.append(fitness)
                all_errors.append(error)
                
                # Update solusi terbaik global
                if error < self.best_error:
                    self.best_solution = solution.copy()
                    self.best_error = error
                    self.best_fitness = fitness
            
            # Fase Update Feromon
            self.update_pheromone(all_solutions, all_fitness)
            
            # Kriteria Penghentian (Stopping Criteria)
            if self.best_error < 0.01:
                if verbose:
                    print(f"Konvergensi di iterasi {iteration+1}. Error: {self.best_error:.6f}")
                break
        
        return self.best_solution, self.best_error

# ============================================================================
# MULTIPLE RUNS DENGAN BERBAGAI PARAMETER (10 KONFIGURASI)
# ============================================================================

def run_multiple_experiments(n_runs=10):
    """Jalankan ACO dengan 10 kombinasi parameter terpilih"""
    
    print("="*80)
    print("EKSPERIMEN ACO - 10 KONFIGURASI PARAMETER")
    print("="*80)
    print(f"\nMenjalankan {n_runs} eksperimen dengan berbagai konfigurasi parameter...\n")
    
    # Definisi 10 variasi parameter yang representatif
    param_configs = [
        # Config 1-3: Variasi jumlah semut (rendah, sedang, tinggi)
        {'n_ants': 20, 'n_iterations': 200, 'evaporation_rate': 0.1},
        {'n_ants': 40, 'n_iterations': 200, 'evaporation_rate': 0.1},
        {'n_ants': 60, 'n_iterations': 200, 'evaporation_rate': 0.1},
        
        # Config 4-6: Variasi evaporation rate (rendah, sedang, tinggi)
        {'n_ants': 30, 'n_iterations': 200, 'evaporation_rate': 0.05},
        {'n_ants': 30, 'n_iterations': 200, 'evaporation_rate': 0.15},
        {'n_ants': 30, 'n_iterations': 200, 'evaporation_rate': 0.25},
        
        # Config 7-9: Variasi iterasi (rendah, sedang, tinggi)
        {'n_ants': 30, 'n_iterations': 150, 'evaporation_rate': 0.1},
        {'n_ants': 30, 'n_iterations': 300, 'evaporation_rate': 0.1},
        {'n_ants': 30, 'n_iterations': 400, 'evaporation_rate': 0.1},
        
        # Config 10: Kombinasi optimal (berdasarkan teori)
        {'n_ants': 50, 'n_iterations': 250, 'evaporation_rate': 0.15},
    ]
    
    results_list = []
    
    start_time = time.time()
    
    for run_id in range(n_runs):
        config = param_configs[run_id]
        
        # Set random seed untuk reproducibility
        np.random.seed(run_id)
        random.seed(run_id)
        
        # Jalankan ACO
        aco = AntColonyOptimizer(
            n_ants=config['n_ants'],
            n_iterations=config['n_iterations'],
            evaporation_rate=config['evaporation_rate']
        )
        
        run_start = time.time()
        best_solution, best_error = aco.optimize()
        run_time = time.time() - run_start
        
        # Hitung selisih dengan solusi asli
        diff = np.abs(best_solution - true_solution)
        max_diff = np.max(diff)
        avg_diff = np.mean(diff)
        
        # Simpan hasil
        results_list.append({
            'Run': run_id + 1,
            'N_Ants': config['n_ants'],
            'N_Iterations': config['n_iterations'],
            'Evaporation_Rate': config['evaporation_rate'],
            'Best_Error': best_error,
            'Max_Diff': max_diff,
            'Avg_Diff': avg_diff,
            'Time_Seconds': run_time,
            'Solution': best_solution.tolist()
        })
        
        # Progress indicator
        print(f"✓ Run {run_id + 1}/{n_runs} selesai | "
              f"Error: {best_error:.6f} | "
              f"Time: {run_time:.2f}s")
    
    total_time = time.time() - start_time
    print(f"\n⏱ Total waktu: {total_time:.2f} detik")
    
    return pd.DataFrame(results_list)

# ============================================================================
# ANALISIS HASIL
# ============================================================================

def analyze_results(df):
    """Analisis mendalam hasil eksperimen"""
    
    print("\n" + "="*80)
    print("ANALISIS HASIL EKSPERIMEN")
    print("="*80)
    
    # 1. STATISTIK UMUM
    print("\n📊 STATISTIK UMUM:")
    print("-" * 80)
    print(f"Total Runs              : {len(df)}")
    print(f"Best Error              : {df['Best_Error'].min():.6f}")
    print(f"Worst Error             : {df['Best_Error'].max():.6f}")
    print(f"Average Error           : {df['Best_Error'].mean():.6f}")
    print(f"Median Error            : {df['Best_Error'].median():.6f}")
    print(f"Std Dev Error           : {df['Best_Error'].std():.6f}")
    
    # 2. SEMUA RUNS (DIURUTKAN DARI TERBAIK)
    print("\n🏆 HASIL SEMUA RUNS (URUT BERDASARKAN ERROR):")
    print("-" * 80)
    sorted_df = df.sort_values('Best_Error')
    for idx, row in sorted_df.iterrows():
        print(f"Run {row['Run']:2d} | Error: {row['Best_Error']:10.6f} | "
              f"Ants: {row['N_Ants']:2.0f} | Iter: {row['N_Iterations']:3.0f} | "
              f"Evap: {row['Evaporation_Rate']:.2f} | Time: {row['Time_Seconds']:.2f}s")
    
    # 3. ANALISIS PARAMETER: N_ANTS
    print("\n🐜 PENGARUH JUMLAH SEMUT:")
    print("-" * 80)
    ants_analysis = df.groupby('N_Ants')['Best_Error'].agg(['mean', 'min', 'max'])
    ants_analysis = ants_analysis.sort_values('mean')
    print(ants_analysis.to_string())
    
    # 4. ANALISIS PARAMETER: EVAPORATION_RATE
    print("\n💨 PENGARUH EVAPORATION RATE:")
    print("-" * 80)
    evap_analysis = df.groupby('Evaporation_Rate')['Best_Error'].agg(['mean', 'min', 'max'])
    evap_analysis = evap_analysis.sort_values('mean')
    print(evap_analysis.to_string())
    
    # 5. ANALISIS PARAMETER: N_ITERATIONS
    print("\n🔄 PENGARUH JUMLAH ITERASI:")
    print("-" * 80)
    iter_analysis = df.groupby('N_Iterations')['Best_Error'].agg(['mean', 'min', 'max'])
    iter_analysis = iter_analysis.sort_values('mean')
    print(iter_analysis.to_string())
    
    # 6. REKOMENDASI PARAMETER TERBAIK
    print("\n⭐ REKOMENDASI PARAMETER:")
    print("-" * 80)
    best_run = df.loc[df['Best_Error'].idxmin()]
    print(f"Berdasarkan hasil terbaik (Run {best_run['Run']}):")
    print(f"  - Jumlah Semut          : {best_run['N_Ants']:.0f}")
    print(f"  - Jumlah Iterasi        : {best_run['N_Iterations']:.0f}")
    print(f"  - Evaporation Rate      : {best_run['Evaporation_Rate']:.2f}")
    print(f"  - Error yang dicapai    : {best_run['Best_Error']:.6f}")
    print(f"  - Waktu komputasi       : {best_run['Time_Seconds']:.2f} detik")
    
    # 7. SOLUSI TERBAIK VS SOLUSI ASLI
    print("\n🎯 SOLUSI TERBAIK:")
    print("-" * 80)
    best_solution = np.array(best_run['Solution'])
    print(f"{'Var':<5} {'ACO':<12} {'True':<12} {'Diff':<12} {'Status'}")
    print("-" * 80)
    for i in range(8):
        diff = abs(best_solution[i] - true_solution[i])
        status = "✓" if diff < 0.5 else "✗"
        print(f"X{i+1:<4} {best_solution[i]:<12.4f} {true_solution[i]:<12.4f} "
              f"{diff:<12.4f} {status}")
    
    # 8. DISTRIBUSI KUALITAS SOLUSI
    print("\n📈 DISTRIBUSI KUALITAS SOLUSI:")
    print("-" * 80)
    excellent = len(df[df['Best_Error'] < 1])
    good = len(df[(df['Best_Error'] >= 1) & (df['Best_Error'] < 10)])
    ok = len(df[(df['Best_Error'] >= 10) & (df['Best_Error'] < 100)])
    poor = len(df[df['Best_Error'] >= 100])
    
    print(f"Excellent (Error < 1)      : {excellent:2d} runs ({excellent/len(df)*100:.1f}%)")
    print(f"Good (1 <= Error < 10)     : {good:2d} runs ({good/len(df)*100:.1f}%)")
    print(f"OK (10 <= Error < 100)     : {ok:2d} runs ({ok/len(df)*100:.1f}%)")
    print(f"Poor (Error >= 100)        : {poor:2d} runs ({poor/len(df)*100:.1f}%)")
    
    # 9. KORELASI PARAMETER
    print("\n🔗 KORELASI PARAMETER DENGAN ERROR:")
    print("-" * 80)
    correlations = df[['N_Ants', 'N_Iterations', 'Evaporation_Rate', 'Best_Error']].corr()['Best_Error']
    print(correlations.to_string())
    
    return best_run

# ============================================================================
# VISUALISASI HASIL
# ============================================================================

def visualize_results(df):
    """Membuat visualisasi lengkap hasil eksperimen"""
    
    print("\n" + "="*80)
    print("MEMBUAT VISUALISASI...")
    print("="*80)
    
    # Create figure dengan multiple subplots
    fig = plt.figure(figsize=(16, 12))
    
    # 1. ERROR SETIAP RUN (Bar Chart)
    ax1 = plt.subplot(3, 3, 1)
    colors = plt.cm.RdYlGn_r(df['Best_Error'] / df['Best_Error'].max())
    bars = ax1.bar(df['Run'], df['Best_Error'], color=colors, edgecolor='black', linewidth=0.5)
    ax1.set_xlabel('Run Number', fontweight='bold')
    ax1.set_ylabel('Best Error', fontweight='bold')
    ax1.set_title('Error per Run', fontweight='bold', fontsize=12)
    ax1.grid(axis='y', alpha=0.3)
    
    # Tambahkan nilai error di atas bar untuk run terbaik
    best_idx = df['Best_Error'].idxmin()
    ax1.text(df.loc[best_idx, 'Run'], df.loc[best_idx, 'Best_Error'], 
             f"{df.loc[best_idx, 'Best_Error']:.2f}", 
             ha='center', va='bottom', fontweight='bold', color='red')
    
    # 2. PENGARUH JUMLAH SEMUT
    ax2 = plt.subplot(3, 3, 2)
    ants_grouped = df.groupby('N_Ants')['Best_Error'].agg(['mean', 'min', 'max']).reset_index()
    ax2.plot(ants_grouped['N_Ants'], ants_grouped['mean'], 'o-', linewidth=2, 
             markersize=8, label='Mean Error', color='blue')
    ax2.fill_between(ants_grouped['N_Ants'], ants_grouped['min'], ants_grouped['max'], 
                      alpha=0.3, label='Min-Max Range')
    ax2.set_xlabel('Number of Ants', fontweight='bold')
    ax2.set_ylabel('Error', fontweight='bold')
    ax2.set_title('Impact of Ant Population', fontweight='bold', fontsize=12)
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. PENGARUH EVAPORATION RATE
    ax3 = plt.subplot(3, 3, 3)
    evap_grouped = df.groupby('Evaporation_Rate')['Best_Error'].agg(['mean', 'min', 'max']).reset_index()
    ax3.plot(evap_grouped['Evaporation_Rate'], evap_grouped['mean'], 'o-', linewidth=2, 
             markersize=8, label='Mean Error', color='green')
    ax3.fill_between(evap_grouped['Evaporation_Rate'], evap_grouped['min'], evap_grouped['max'], 
                      alpha=0.3, label='Min-Max Range')
    ax3.set_xlabel('Evaporation Rate', fontweight='bold')
    ax3.set_ylabel('Error', fontweight='bold')
    ax3.set_title('Impact of Evaporation Rate', fontweight='bold', fontsize=12)
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. PENGARUH JUMLAH ITERASI
    ax4 = plt.subplot(3, 3, 4)
    iter_grouped = df.groupby('N_Iterations')['Best_Error'].agg(['mean', 'min', 'max']).reset_index()
    ax4.plot(iter_grouped['N_Iterations'], iter_grouped['mean'], 'o-', linewidth=2, 
             markersize=8, label='Mean Error', color='red')
    ax4.fill_between(iter_grouped['N_Iterations'], iter_grouped['min'], iter_grouped['max'], 
                      alpha=0.3, label='Min-Max Range')
    ax4.set_xlabel('Number of Iterations', fontweight='bold')
    ax4.set_ylabel('Error', fontweight='bold')
    ax4.set_title('Impact of Iterations', fontweight='bold', fontsize=12)
    ax4.legend()
    ax4.grid(alpha=0.3)
    
    # 5. WAKTU KOMPUTASI
    ax5 = plt.subplot(3, 3, 5)
    ax5.scatter(df['Best_Error'], df['Time_Seconds'], c=df['N_Ants'], 
                cmap='viridis', s=100, edgecolor='black', linewidth=0.5)
    ax5.set_xlabel('Best Error', fontweight='bold')
    ax5.set_ylabel('Time (seconds)', fontweight='bold')
    ax5.set_title('Error vs Computation Time', fontweight='bold', fontsize=12)
    ax5.set_xscale('log')
    cbar = plt.colorbar(ax5.collections[0], ax=ax5)
    cbar.set_label('N_Ants', fontweight='bold')
    ax5.grid(alpha=0.3)
    
    # 6. HEATMAP PARAMETER vs ERROR
    ax6 = plt.subplot(3, 3, 6)
    pivot_data = df.pivot_table(values='Best_Error', 
                                  index='N_Ants', 
                                  columns='Evaporation_Rate', 
                                  aggfunc='mean')
    sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap='RdYlGn_r', 
                ax=ax6, cbar_kws={'label': 'Error'})
    ax6.set_title('Heatmap: Ants vs Evaporation', fontweight='bold', fontsize=12)
    ax6.set_ylabel('Number of Ants', fontweight='bold')
    ax6.set_xlabel('Evaporation Rate', fontweight='bold')
    
    # 7. DISTRIBUSI ERROR (HISTOGRAM)
    ax7 = plt.subplot(3, 3, 7)
    ax7.hist(df['Best_Error'], bins=8, color='skyblue', edgecolor='black', alpha=0.7)
    ax7.axvline(df['Best_Error'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {df["Best_Error"].mean():.2f}')
    ax7.axvline(df['Best_Error'].median(), color='green', linestyle='--', 
                linewidth=2, label=f'Median: {df["Best_Error"].median():.2f}')
    ax7.set_xlabel('Best Error', fontweight='bold')
    ax7.set_ylabel('Frequency', fontweight='bold')
    ax7.set_title('Error Distribution', fontweight='bold', fontsize=12)
    ax7.legend()
    ax7.grid(axis='y', alpha=0.3)
    
    # 8. SOLUSI TERBAIK vs SOLUSI ASLI
    ax8 = plt.subplot(3, 3, 8)
    best_run = df.loc[df['Best_Error'].idxmin()]
    best_solution = np.array(best_run['Solution'])
    x_vars = np.arange(1, 9)
    
    width = 0.35
    ax8.bar(x_vars - width/2, true_solution, width, label='True Solution', 
            color='green', alpha=0.7, edgecolor='black')
    ax8.bar(x_vars + width/2, best_solution, width, label='ACO Solution', 
            color='orange', alpha=0.7, edgecolor='black')
    
    ax8.set_xlabel('Variable (X)', fontweight='bold')
    ax8.set_ylabel('Value', fontweight='bold')
    ax8.set_title('Best Solution Comparison', fontweight='bold', fontsize=12)
    ax8.set_xticks(x_vars)
    ax8.set_xticklabels([f'X{i}' for i in x_vars])
    ax8.legend()
    ax8.grid(axis='y', alpha=0.3)
    
    # 9. PARAMETER CONFIGURATION PERFORMANCE
    ax9 = plt.subplot(3, 3, 9)
    df_sorted = df.sort_values('Best_Error')
    config_labels = [f"Run {row['Run']}\n({row['N_Ants']:.0f},{row['N_Iterations']:.0f},{row['Evaporation_Rate']:.2f})" 
                     for _, row in df_sorted.iterrows()]
    
    colors_sorted = plt.cm.RdYlGn_r(df_sorted['Best_Error'] / df_sorted['Best_Error'].max())
    ax9.barh(range(len(df_sorted)), df_sorted['Best_Error'], color=colors_sorted, 
             edgecolor='black', linewidth=0.5)
    ax9.set_yticks(range(len(df_sorted)))
    ax9.set_yticklabels(config_labels, fontsize=7)
    ax9.set_xlabel('Best Error', fontweight='bold')
    ax9.set_title('Ranked Configuration Performance', fontweight='bold', fontsize=12)
    ax9.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('aco_visualization.png', dpi=300, bbox_inches='tight')
    print("✓ Visualisasi disimpan ke: aco_visualization.png")
    plt.show()
    
    # ========================================================================
    # VISUALISASI TAMBAHAN: DETAIL SOLUSI TOP 3
    # ========================================================================
    
    fig2, axes = plt.subplots(1, 3, figsize=(15, 5))
    top3 = df.nsmallest(3, 'Best_Error')
    
    for idx, (ax, (_, row)) in enumerate(zip(axes, top3.iterrows())):
        solution = np.array(row['Solution'])
        x_vars = np.arange(1, 9)
        
        diff = np.abs(solution - true_solution)
        colors_diff = ['green' if d < 0.5 else 'orange' if d < 1 else 'red' for d in diff]
        
        width = 0.35
        ax.bar(x_vars - width/2, true_solution, width, label='True', 
               color='green', alpha=0.5, edgecolor='black')
        ax.bar(x_vars + width/2, solution, width, label='ACO', 
               color=colors_diff, alpha=0.7, edgecolor='black')
        
        ax.set_xlabel('Variable', fontweight='bold')
        ax.set_ylabel('Value', fontweight='bold')
        ax.set_title(f'Rank #{idx+1} - Run {row["Run"]}\nError: {row["Best_Error"]:.4f}', 
                     fontweight='bold')
        ax.set_xticks(x_vars)
        ax.set_xticklabels([f'X{i}' for i in x_vars])
        ax.legend()
        ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('aco_top3_solutions.png', dpi=300, bbox_inches='tight')
    print("✓ Top 3 solutions disimpan ke: aco_top3_solutions.png")
    plt.show()
    
    print("\n✅ Semua visualisasi selesai dibuat!")

# ============================================================================
# JALANKAN EKSPERIMEN
# ============================================================================

if _name_ == "_main_":
    # Jalankan 10 eksperimen
    df_results = run_multiple_experiments(n_runs=10)
    
    # Analisis hasil
    best_config = analyze_results(df_results)
    
    # Visualisasi hasil
    visualize_results(df_results)
    
    # Simpan hasil ke CSV (opsional)
    df_results.to_csv('aco_results_10.csv', index=False)
    print("\n💾 Hasil disimpan ke: aco_results_10.csv")
    
    print("\n" + "="*80)
    print("EKSPERIMEN SELESAI!")
    print("="*80)

EKSPERIMEN ACO - 10 KONFIGURASI PARAMETER

Menjalankan 10 eksperimen dengan berbagai konfigurasi parameter...



TypeError: AntColonyOptimizer() takes no arguments